In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

%matplotlib inline
%load_ext autoreload
%autoreload 2
from creation_functions import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
df_P = pd.io.parsers.read_csv(filepath_or_buffer = 'data/cham3_coll.csv',index_col = 0)
print df_P[:5]

                       UnixTime  e2v03  no2  Temp    Rh
date                                                   
2014-07-17 17:55:52  1405619752    608  874  36.2  23.3
2014-07-17 17:55:57  1405619757    608  874  36.1  23.3
2014-07-17 17:56:02  1405619762    608  873  36.2  23.3
2014-07-17 17:56:06  1405619766    608  873  36.2  23.2
2014-07-17 17:56:11  1405619771    608  873  36.2  23.2


In [66]:
df_P.drop(df_P.columns[0], axis=1, inplace=True)
print df_P.head()

                     e2v03  no2  Temp    Rh
date                                       
2014-07-17 17:55:52    608  874  36.2  23.3
2014-07-17 17:55:57    608  874  36.1  23.3
2014-07-17 17:56:02    608  873  36.2  23.3
2014-07-17 17:56:06    608  873  36.2  23.2
2014-07-17 17:56:11    608  873  36.2  23.2


## Make log, slope, and integral terms

In [67]:
#log terms
df_P['ln_O3'] = np.log(df_P['e2v03'])
df_P['ln_temp'] =np.log(df_P['Temp'])
df_P['ln_rh'] = np.log(df_P['Rh'])

feature_list = ['e2v03', 'ln_O3', 'Temp', 'ln_temp', 'Rh', 'ln_rh']
print feature_list

['e2v03', 'ln_O3', 'Temp', 'ln_temp', 'Rh', 'ln_rh']


In [68]:
for i in range(0,len(feature_list)):
    print i
    df_P = make_func_caller_find_lag_integral(df_P, 1, 5, 2, feature_list[i])
    df_P = make_func_caller_find_lead_integral(df_P, 1, 5, 2, feature_list[i])
    df_P = make_func_caller_find_lag_slope(df_P, 1, 5, 2, feature_list[i])
    df_P = make_func_caller_find_lead_slope(df_P, 1, 5, 2, feature_list[i])
    df_P = make_func_caller_find_lag_integral(df_P, 1, 5, 2, feature_list[i])
    df_P = make_func_caller_find_lead_integral(df_P, 1, 5, 2, feature_list[i])
    df_P = make_func_caller_find_lag_slope(df_P, 1, 5, 2, feature_list[i])
    df_P = make_func_caller_find_lead_slope(df_P, 1, 5, 2, feature_list[i])

features = list(df_P.columns)
print features

0
1
2
3
4
5
['e2v03', 'no2', 'Temp', 'Rh', 'ln_O3', 'ln_temp', 'ln_rh', 'e2v03_int_lag_1', 'e2v03_int_lag_3', 'e2v03_int_lag_5', 'e2v03_int_lead_1', 'e2v03_int_lead_3', 'e2v03_int_lead_5', 'diff', 'e2v03_slope_lag_1', 'e2v03_slope_lag_3', 'e2v03_slope_lag_5', 'e2v03_slope_lead_1', 'e2v03_slope_lead_3', 'e2v03_slope_lead_5', 'ln_O3_int_lag_1', 'ln_O3_int_lag_3', 'ln_O3_int_lag_5', 'ln_O3_int_lead_1', 'ln_O3_int_lead_3', 'ln_O3_int_lead_5', 'ln_O3_slope_lag_1', 'ln_O3_slope_lag_3', 'ln_O3_slope_lag_5', 'ln_O3_slope_lead_1', 'ln_O3_slope_lead_3', 'ln_O3_slope_lead_5', 'Temp_int_lag_1', 'Temp_int_lag_3', 'Temp_int_lag_5', 'Temp_int_lead_1', 'Temp_int_lead_3', 'Temp_int_lead_5', 'Temp_slope_lag_1', 'Temp_slope_lag_3', 'Temp_slope_lag_5', 'Temp_slope_lead_1', 'Temp_slope_lead_3', 'Temp_slope_lead_5', 'ln_temp_int_lag_1', 'ln_temp_int_lag_3', 'ln_temp_int_lag_5', 'ln_temp_int_lead_1', 'ln_temp_int_lead_3', 'ln_temp_int_lead_5', 'ln_temp_slope_lag_1', 'ln_temp_slope_lag_3', 'ln_temp_slope_lag_

In [79]:
df_P.shape

(77884, 80)

In [70]:
#Delete the first and last 90 rows of the dataframe to remove all NaNs.
df_P = df_P.ix[10:len(df_P['e2v03'])-10]

#### Make polynomial and interaction features

In [87]:
X = df_P[feature_list].values
poly = PolynomialFeatures(2)
features = poly.fit_transform(X)

#### See which features are combined by poly

In [107]:
poly.powers_[1,0]

1

In [119]:
name = []
i_count = 0
for i in range(0,len(poly.powers_)):
    if np.sum(poly.powers_[i]) > 1:
        for j in range(0,len(feature_list)):
            if poly.powers_[i,j] == 2:
                name.append(feature_list[j] + '_sq')
            if poly.powers_[i,j] == 1:
                i_count += 1
                if i_count == 1:
                    name1 = feature_list[j]
                elif i_count == 2:
                    name.append(name1 + '_interact_' +feature_list[j])
                    i_count = 0
    else: 
        name.append(0)
        

In [142]:
poly_feat = []
poly_data = []
for i in range(0,len(poly.powers_)):
    if name[i] != 0:
        poly_feat.append(name[i])
        df_P[name[i]] = features[:,i]
        poly_data.append(features[i])

In [143]:
df_P.head()

,e2v03,no2,Temp,Rh,ln_O3,ln_temp,ln_rh,e2v03_int_lag_1,e2v03_int_lag_3,e2v03_int_lag_5,...,Temp_sq,Temp_interact_ln_temp,Temp_interact_Rh,Temp_interact_ln_rh,ln_temp_sq,ln_temp_interact_Rh,ln_temp_interact_ln_rh,Rh_sq,Rh_interact_ln_rh,ln_rh_sq
date,,,,,,,,,,,,,,,,,,,,,
2014-07-17 17:56:40,608,871,36.2,23.4,6.410175,3.589059,3.152736,0,1218,2434.5,...,1310.44,129.923940,847.08,114.129044,12.881345,83.983983,11.315356,547.56,73.774023,9.939744
2014-07-17 17:56:45,609,870,36.2,23.3,6.411818,3.589059,3.148453,0,1217.5,2435,...,1310.44,129.923940,843.46,113.974012,12.881345,83.625077,11.299985,542.89,73.358963,9.912759
2014-07-17 17:56:50,609,870,36.2,23.3,6.411818,3.589059,3.148453,0,1217,2435,...,1310.44,129.923940,843.46,113.974012,12.881345,83.625077,11.299985,542.89,73.358963,9.912759
2014-07-17 17:56:55,608,869,36.3,23.3,6.410175,3.591818,3.148453,0,1217.5,2435,...,1317.69,130.382984,845.79,114.288857,12.901155,83.689353,11.308671,542.89,73.358963,9.912759
2014-07-17 17:57:00,609,868,36.2,23.2,6.411818,3.589059,3.144152,0,1217.5,2434.5,...,1310.44,129.923940,839.84,113.818312,12.881345,83.266172,11.284548,538.24,72.944333,9.885694


In [145]:
df_P.to_csv(path_or_buf = 'cham3_coll_feat.csv')